In [ ]:
!pip install pretty_midi

In [ ]:
from collections import defaultdict
import copy
from math import log, floor, ceil
import pprint
import matplotlib.pyplot as plt
import pretty_midi
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage
import numpy as np
import random

In [ ]:
DEBUG = False

# The MIDI pitches we use.
PITCHES = range(24,108)
OFFSET = 109-21
PITCHES_MAP = { p : i for i, p in enumerate(PITCHES) }
print(len(PITCHES))

84


In [ ]:
def nearest_pow2(x):
    '''Normalize input to nearest power of 2, or midpoints between
    consecutive powers of two. Round down when halfway between two
    possibilities.'''

    low = 2**int(floor(log(x, 2)))
    high = 2**int(ceil(log(x, 2)))
    mid = (low + high) / 2

    if x < mid:
        high = mid
    else:
        low = mid
    if high - x < x - low:
        nearest = high
    else:
        nearest = low
    return nearest

In [ ]:
def get_note_track(mid_file):
    '''Given a MIDI object, return the first track with note events.'''
    midi_tracks = mid_file.tracks
    for i, track in enumerate(midi_tracks):
        for msg in track:
            if msg.type == 'note_on':
                return i, track
    raise ValueError(
        'MIDI object does not contain any tracks with note messages.')

def quantize_tick(tick, ticks_per_quarter, quantization):
    '''Quantize the timestamp or tick.
    Arguments:
    tick -- An integer timestamp
    ticks_per_quarter -- The number of ticks per quarter note
    quantization -- The note duration, represented as 1/2**quantization
    '''
    assert (ticks_per_quarter * 4) % 2 ** quantization == 0, \
        'Quantization too fine. Ticks per quantum must be an integer.'
    ticks_per_quantum = (ticks_per_quarter * 4) / float(2 ** quantization)
    quantized_ticks = int(
        round(tick / float(ticks_per_quantum)) * ticks_per_quantum)
    return quantized_ticks

def unquantize(mid, style_mid):
    unquantized_mid = copy.deepcopy(mid)
    # By convention, Track 0 contains metadata and Track 1 contains
    # the note on and note off events.
    orig_note_track_idx, orig_note_track = get_note_track(mid)
    style_note_track_idx, style_note_track = get_note_track(style_mid)

    note_track = unquantize_track(orig_note_track, style_note_track)
    unquantized_mid.tracks[orig_note_track_idx] = note_track

    return unquantized_mid

def unquantize_track(orig_track, style_track):
    '''Returns the unquantised orig_track with encoded velocities from the style_track.
    Arguments:
    orig_track -- Non-quantised MIDI object
    style_track -- Quantised and stylised MIDI object '''

    first_note_msg_idx = None

    for i, msg in enumerate(orig_track):
        if msg.type == 'note_on':
            orig_first_note_msg_idx = i
            break

    for i, msg in enumerate(style_track):
        if msg.type == 'note_on':
            style_first_note_msg_idx = i
            break

    orig_cum_msgs = zip(
        np.cumsum([msg.time for msg in orig_track[orig_first_note_msg_idx:]]),
        [msg for msg in orig_track[orig_first_note_msg_idx:]])

    style_cum_msgs = zip(
        np.cumsum([msg.time for msg in style_track[style_first_note_msg_idx:]]),
        [msg for msg in style_track[style_first_note_msg_idx:]])

    # orig_cum_msgs.sort(key=lambda (cum_time, msg): cum_time)
    # style_cum_msgs.sort(key=lambda (cum_time, msg): cum_time)
    orig_cum_msgs.sort(key=lambda x: x[0])
    style_cum_msgs.sort(key=lambda x: x[0])

    open_msgs = defaultdict(list)

    for cum_time, msg in orig_cum_msgs:
        if msg.type == 'note_on' and msg.velocity > 0:
            open_msgs[msg.note].append((cum_time,msg))

    for i, (cum_time, msg) in enumerate(style_cum_msgs):
         if msg.type == 'note_on' and msg.velocity > 0:
            note_on_open_msgs = open_msgs[msg.note]
            note_on_cum_time, note_on_msg = note_on_open_msgs[0]
            note_on_msg.velocity = msg.velocity
            open_msgs[msg.note] = note_on_open_msgs[1:]

    return orig_track

def quantize(mid, quantization=5):
    '''Return a midi object whose notes are quantized to
    1/2**quantization notes.
    Arguments:
    mid -- MIDI object
    quantization -- The note duration, represented as
      1/2**quantization.'''

    quantized_mid = copy.deepcopy(mid)
    # By convention, Track 0 contains metadata and Track 1 contains
    # the note on and note off events.
    note_track_idx, note_track = get_note_track(mid)

    new_track = quantize_track( note_track, mid.ticks_per_beat, quantization)
    if new_track == None:
        return None
    quantized_mid.tracks[note_track_idx] = new_track
    return quantized_mid

def quantize_track(track, ticks_per_quarter, quantization):
    '''Return the differential time stamps of the note_on, note_off, and
    end_of_track events, in order of appearance, with the note_on events
    quantized to the grid given by the quantization.
    Arguments:
    track -- MIDI track containing note event and other messages
    ticks_per_quarter -- The number of ticks per quarter note
    quantization -- The note duration, represented as
      1/2**quantization.'''

    pp = pprint.PrettyPrinter()

    # Message timestamps are represented as differences between
    # consecutive events. Annotate messages with cumulative timestamps.

    # Assume the following structure:
    # [header meta messages] [note messages] [end_of_track message]
    first_note_msg_idx = None
    for i, msg in enumerate(track):
        if msg.type == 'note_on':
            first_note_msg_idx = i
            break

    cum_msgs = list(zip(
        np.cumsum([msg.time for msg in track[first_note_msg_idx:]]),
        [msg for msg in track[first_note_msg_idx:]]))
    

    end_of_track_cum_time = cum_msgs[-1][0]
 
    quantized_track = MidiTrack()
    quantized_track.extend(track[:first_note_msg_idx])
    # Keep track of note_on events that have not had an off event yet.
    # note number -> message
    open_msgs = defaultdict(list)
    quantized_msgs = []
    for cum_time, msg in cum_msgs:
        if DEBUG:
            print('Message:', msg)
            print ('Open messages:')
            pp.pprint(open_msgs)
        if msg.type == 'note_on' and msg.velocity > 0:
            # Store until note off event. Note that there can be
            # several note events for the same note. Subsequent
            # note_off events will be associated with these note_on
            # events in FIFO fashion.
            open_msgs[msg.note].append((cum_time, msg))
        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            # assert msg.note in open_msgs, \
            #     'Bad MIDI. Cannot have note off event before note on event'

            if msg.note not in open_msgs:
                 print('Bad MIDI. Cannot have note off event before note on event')
                 return
            
            note_on_open_msgs = open_msgs[msg.note]

            if len(note_on_open_msgs) == 0:

                print('Bad MIDI, Note has no end time.')
                return

            # assert len(note_on_open_msgs) > 0, 'Bad MIDI, Note has no end time.'

            note_on_cum_time, note_on_msg = note_on_open_msgs[0]
            open_msgs[msg.note] = note_on_open_msgs[1:]

            # Quantized note_on time
            quantized_note_on_cum_time = quantize_tick(
                note_on_cum_time, ticks_per_quarter, quantization)

            # The cumulative time of note_off is the quantized
            # cumulative time of note_on plus the orginal difference
            # of the unquantized cumulative times.
            quantized_note_off_cum_time = quantized_note_on_cum_time + (cum_time - note_on_cum_time)
            quantized_msgs.append((min(end_of_track_cum_time, quantized_note_on_cum_time), note_on_msg))
            quantized_msgs.append((min(end_of_track_cum_time, quantized_note_off_cum_time), msg))

            if DEBUG:
                print('Appended', quantized_msgs[-2:])
        elif msg.type == 'end_of_track':
            quantized_msgs.append((cum_time, msg))

        if DEBUG:
            print('\n')

    # Now, sort the quantized messages by (cumulative time,
    # note_type), making sure that note_on events come before note_off
    # events when two event have the same cumulative time. Compute
    # differential times and construct the quantized track messages.
    
    # quantized_msgs.sort(
    #     key=lambda (cum_time, msg): cum_time
    #     if (msg.type=='note_on' and msg.velocity > 0) else cum_time + 0.5)

    quantized_msgs.sort(key=lambda x: x[0])
    # for x in quantized_msgs:
    #   msg = x[1]                       ???????
    #   if not(msg.type=='note_on' and msg.velocity > 0):
    #     x[0] += 0.5 

    diff_times = [quantized_msgs[0][0]] + list(
        np.diff([ msg[0] for msg in quantized_msgs ]))
    for diff_time, (cum_time, msg) in zip(diff_times, quantized_msgs):
        quantized_track.append(msg.copy(time=diff_time))
    if DEBUG:
        print('Quantized messages:')
        pp.pprint(quantized_msgs)
        pp.pprint(diff_times)
    return quantized_track

In [ ]:
def midi_to_array_one_hot(mid, quantization):
    '''Return array representation of a 4/4 time signature, MIDI object.
    Normalize the number of time steps in track to a power of 2. Then
    construct a T x N*2 array A (T = number of time steps, N = number of
    MIDI note numbers) where [A(t,n), A(t, n+1)] is the state of the note number
    at time step t.
    Arguments:
    mid -- MIDI object with a 4/4 time signature
    quantization -- The note duration, represented as 1/2**quantization.'''

    time_sig_msgs = [ msg for msg in mid.tracks[0] if msg.type == 'time_signature' ]
    assert len(time_sig_msgs) == 1, 'No time signature found'
    time_sig = time_sig_msgs[0]
    assert time_sig.numerator == 4 and time_sig.denominator == 4, 'Not 4/4 time.'

    # Quantize the notes to a grid of time steps.
    mid = quantize(mid, quantization=quantization)
    
    # Convert the note timing and velocity to an array.
    _, track = get_note_track(mid)
    ticks_per_quarter = mid.ticks_per_beat
    time_msgs = [msg for msg in track if hasattr(msg, 'time')]
    cum_times = np.cumsum([msg.time for msg in time_msgs])

    track_len_ticks = cum_times[-1]
    if DEBUG:
        print('Track len in ticks:', track_len_ticks)
    notes = [
        (int(time * (2**quantization/4) / (ticks_per_quarter)), msg.type, msg.note, msg.velocity)
        for (time, msg) in zip(cum_times, time_msgs)
        if msg.type == 'note_on' or msg.type == 'note_off']

    num_steps = int(round(track_len_ticks / float(ticks_per_quarter)*2**quantization/4))
    normalized_num_steps = int(nearest_pow2(num_steps))
    print(normalized_num_steps)
    
    # notes.sort(key=lambda (position, note_type, note_num, velocity):(position,-velocity))

    notes.sort(key= lambda x: x[-1], reverse=True)  # sort on secondary key
    notes.sort(key= lambda x: x[0])   # sort on primary key

    if DEBUG:
        # pp = pprint.PrettyPrinter()
        print(num_steps)
        print(normalized_num_steps)
        # pp.pprint(notes)

    # midi_array = np.zeros((normalized_num_steps, len(PITCHES)*2))
    midi_array = np.zeros((normalized_num_steps, len(PITCHES), 2))
    velocity_array = np.zeros((normalized_num_steps, len(PITCHES)))
    open_msgs = defaultdict(list)

    for (position, note_type, note_num, velocity) in notes:
        if position == normalized_num_steps:
            # print 'Warning: truncating from position {} to {}'.format(position, normalized_num_steps - 1)
            position = normalized_num_steps - 1
            # continue

        if position > normalized_num_steps:
            # print 'Warning: skipping note at position {} (greater than {})'.format(position, normalized_num_steps)
            continue

        if note_type == "note_on" and velocity > 0:
            open_msgs[note_num].append((position, note_type, note_num, velocity))
            # midi_array[position, 2*PITCHES_MAP[note_num]] = 1
            # midi_array[position, 2*PITCHES_MAP[note_num]+1] = 1
            midi_array[position, PITCHES_MAP[note_num],0] = 1
            midi_array[position, PITCHES_MAP[note_num],1] = 1
            velocity_array[position, PITCHES_MAP[note_num]] = velocity
        elif note_type == 'note_off' or (note_type == 'note_on' and velocity == 0):

            note_on_open_msgs = open_msgs[note_num]

            if len(note_on_open_msgs) == 0:
                print('Bad MIDI, Note has no end time.')
                return

            stack_pos, _, _, vel = note_on_open_msgs[0]
            open_msgs[note_num] = note_on_open_msgs[1:]
            current_pos = position
            while current_pos > stack_pos:
                # if midi_array[position, PITCHES_MAP[note_num]] != 1:
                # midi_array[current_pos, 2*PITCHES_MAP[note_num]] = 0
                # midi_array[current_pos, 2*PITCHES_MAP[note_num]+1] = 1
                midi_array[current_pos, PITCHES_MAP[note_num],0] = 0
                midi_array[current_pos, PITCHES_MAP[note_num],1] = 1
                velocity_array[current_pos, PITCHES_MAP[note_num]] = vel
                current_pos -= 1

    for (position, note_type, note_num, velocity) in notes:
        if position == normalized_num_steps:
            print('Warning: truncating from position {} to {}'.format(position, normalized_num_steps - 1))
            position = normalized_num_steps - 1
            # continue

        if position > normalized_num_steps:
            # print 'Warning: skipping note at position {} (greater than {})'.format(position, normalized_num_steps)
            continue
        if note_type == "note_on" and velocity > 0:
            open_msgs[note_num].append((position, note_type, note_num, velocity))
            # midi_array[position, 2*PITCHES_MAP[note_num]] = 1
            # midi_array[position, 2*PITCHES_MAP[note_num]+1] = 1
            midi_array[position, PITCHES_MAP[note_num],0] = 1
            midi_array[position, PITCHES_MAP[note_num],1] = 1
            velocity_array[position, PITCHES_MAP[note_num]] = velocity

    assert len(midi_array) == len(velocity_array)
    return midi_array, velocity_array

In [ ]:
def midi_to_array_one_hot_justnote(mid, quantization):
    '''Return array representation of a 4/4 time signature, MIDI object.
    Normalize the number of time steps in track to a power of 2. Then
    construct a T x N*2 array A (T = number of time steps, N = number of
    MIDI note numbers) where [A(t,n), A(t, n+1)] is the state of the note number
    at time step t.
    Arguments:
    mid -- MIDI object with a 4/4 time signature
    quantization -- The note duration, represented as 1/2**quantization.'''

    time_sig_msgs = [ msg for msg in mid.tracks[0] if msg.type == 'time_signature' ]
    assert len(time_sig_msgs) == 1, 'No time signature found'
    time_sig = time_sig_msgs[0]
    assert time_sig.numerator == 4 and time_sig.denominator == 4, 'Not 4/4 time.'

    # Quantize the notes to a grid of time steps.
    mid = quantize(mid, quantization=quantization)
    
    # Convert the note timing and velocity to an array.
    _, track = get_note_track(mid)
    ticks_per_quarter = mid.ticks_per_beat
    time_msgs = [msg for msg in track if hasattr(msg, 'time')]
    cum_times = np.cumsum([msg.time for msg in time_msgs])

    track_len_ticks = cum_times[-1]
    if DEBUG:
        print('Track len in ticks:', track_len_ticks)
    notes = [
        (int(time * (2**quantization/4) / (ticks_per_quarter)), msg.type, msg.note, msg.velocity)
        for (time, msg) in zip(cum_times, time_msgs)
        if msg.type == 'note_on' or msg.type == 'note_off']

    num_steps = int(round(track_len_ticks / float(ticks_per_quarter)*2**quantization/4))
    normalized_num_steps = int(nearest_pow2(num_steps))
    print(normalized_num_steps)
    
    # notes.sort(key=lambda (position, note_type, note_num, velocity):(position,-velocity))

    notes.sort(key= lambda x: x[-1], reverse=True)  # sort on secondary key
    notes.sort(key= lambda x: x[0])   # sort on primary key

    if DEBUG:
        # pp = pprint.PrettyPrinter()
        print(num_steps)
        print(normalized_num_steps)
        # pp.pprint(notes)

    # midi_array = np.zeros((normalized_num_steps, len(PITCHES)*2))
    midi_array = np.zeros((normalized_num_steps, len(PITCHES)))
    velocity_array = np.zeros((normalized_num_steps, len(PITCHES)))
    open_msgs = defaultdict(list)

    for (position, note_type, note_num, velocity) in notes:
        if position == normalized_num_steps:
            # print 'Warning: truncating from position {} to {}'.format(position, normalized_num_steps - 1)
            position = normalized_num_steps - 1
            # continue

        if position > normalized_num_steps:
            # print 'Warning: skipping note at position {} (greater than {})'.format(position, normalized_num_steps)
            continue

        if note_type == "note_on" and velocity > 0:
            open_msgs[note_num].append((position, note_type, note_num, velocity))
            # midi_array[position, 2*PITCHES_MAP[note_num]] = 1
            # midi_array[position, 2*PITCHES_MAP[note_num]+1] = 1
           
            midi_array[position, PITCHES_MAP[note_num]] = 1
            velocity_array[position, PITCHES_MAP[note_num]] = velocity
        elif note_type == 'note_off' or (note_type == 'note_on' and velocity == 0):

            note_on_open_msgs = open_msgs[note_num]

            if len(note_on_open_msgs) == 0:
                print('Bad MIDI, Note has no end time.')
                return

            stack_pos, _, _, vel = note_on_open_msgs[0]
            open_msgs[note_num] = note_on_open_msgs[1:]
            current_pos = position
            while current_pos > stack_pos:
                # if midi_array[position, PITCHES_MAP[note_num]] != 1:
                # midi_array[current_pos, 2*PITCHES_MAP[note_num]] = 0
                # midi_array[current_pos, 2*PITCHES_MAP[note_num]+1] = 1
                midi_array[current_pos, PITCHES_MAP[note_num]] = 1
                velocity_array[current_pos, PITCHES_MAP[note_num]] = vel
                current_pos -= 1

    for (position, note_type, note_num, velocity) in notes:
        if position == normalized_num_steps:
            print('Warning: truncating from position {} to {}'.format(position, normalized_num_steps - 1))
            position = normalized_num_steps - 1
            # continue

        if position > normalized_num_steps:
            # print 'Warning: skipping note at position {} (greater than {})'.format(position, normalized_num_steps)
            continue
        if note_type == "note_on" and velocity > 0:
            open_msgs[note_num].append((position, note_type, note_num, velocity))
            # midi_array[position, 2*PITCHES_MAP[note_num]] = 1
            # midi_array[position, 2*PITCHES_MAP[note_num]+1] = 1
            midi_array[position, PITCHES_MAP[note_num]] = 1
            velocity_array[position, PITCHES_MAP[note_num]] = velocity

    assert len(midi_array) == len(velocity_array)
    return midi_array, velocity_array

In [ ]:
def midi_to_array_3D(mid, quantization):
    '''Return a 3D array representation of a midi :
        -First Dim is time
        -Second Dim is note
        -Third Dim is note_status : 1,0,0 hit, 0,1,0 maintained, 0,0,1 mute.'''

    time_sig_msgs = [ msg for msg in mid.tracks[0] if msg.type == 'time_signature' ]
    assert len(time_sig_msgs) == 1, 'No time signature found'
    time_sig = time_sig_msgs[0]
    assert time_sig.numerator == 4 and time_sig.denominator == 4, 'Not 4/4 time.'

    # Quantize the notes to a grid of time steps.
    mid = quantize(mid, quantization=quantization)
    
    # Convert the note timing and velocity to an array.
    _, track = get_note_track(mid)
    ticks_per_quarter = mid.ticks_per_beat
    time_msgs = [msg for msg in track if hasattr(msg, 'time')]
    cum_times = np.cumsum([msg.time for msg in time_msgs])

    track_len_ticks = cum_times[-1]
    if DEBUG:
        print('Track len in ticks:', track_len_ticks)
    notes = [
        (int(time * (2**quantization/4) / (ticks_per_quarter)), msg.type, msg.note, msg.velocity)
        for (time, msg) in zip(cum_times, time_msgs)
        if msg.type == 'note_on' or msg.type == 'note_off']

    num_steps = int(round(track_len_ticks / float(ticks_per_quarter)*2**quantization/4))
    normalized_num_steps = int(nearest_pow2(num_steps))
    
    
    # notes.sort(key=lambda (position, note_type, note_num, velocity):(position,-velocity))

    notes.sort(key= lambda x: x[-1], reverse=True)  # sort on secondary key
    notes.sort(key= lambda x: x[0])   # sort on primary key

    if DEBUG:
        # pp = pprint.PrettyPrinter()
        print(num_steps)
        print(normalized_num_steps)
        # pp.pprint(notes)

    # midi_array = np.zeros((normalized_num_steps, len(PITCHES)*2))
    midi_array = np.zeros((normalized_num_steps, len(PITCHES), 3))
    
    for i in range(len(midi_array)):
      for j in range(len(midi_array[0])):
        mid_array[i][j][2] = 1

    velocity_array = np.zeros((normalized_num_steps, len(PITCHES)))
    open_msgs = defaultdict(list)

    for (position, note_type, note_num, velocity) in notes:
        if position == normalized_num_steps:
            # print 'Warning: truncating from position {} to {}'.format(position, normalized_num_steps - 1)
            position = normalized_num_steps - 1
            # continue

        if position > normalized_num_steps:
            # print 'Warning: skipping note at position {} (greater than {})'.format(position, normalized_num_steps)
            continue

        if note_type == "note_on" and velocity > 0:
            open_msgs[note_num].append((position, note_type, note_num, velocity))
            # Note is hit

            midi_array[position, PITCHES_MAP[note_num],0] = 1
            midi_array[position, PITCHES_MAP[note_num],1] = 0
            midi_array[position, PITCHES_MAP[note_num],2] = 0
            velocity_array[position, PITCHES_MAP[note_num]] = velocity
        elif note_type == 'note_off' or (note_type == 'note_on' and velocity == 0):

            note_on_open_msgs = open_msgs[note_num]

            if len(note_on_open_msgs) == 0:
                print('Bad MIDI, Note has no end time.')
                return

            stack_pos, _, _, vel = note_on_open_msgs[0]
            open_msgs[note_num] = note_on_open_msgs[1:]
            current_pos = position
            while current_pos > stack_pos:
                # Note is hold
                midi_array[current_pos, PITCHES_MAP[note_num],0] = 0
                midi_array[current_pos, PITCHES_MAP[note_num],1] = 1
                midi_array[current_pos, PITCHES_MAP[note_num],2] = 0
                velocity_array[current_pos, PITCHES_MAP[note_num]] = vel
                current_pos -= 1
        # else:
        #   # Note is mute
        #   midi_array[current_pos, PITCHES_MAP[note_num],0] = 0
        #   midi_array[current_pos, PITCHES_MAP[note_num],1] = 0
        #   midi_array[current_pos, PITCHES_MAP[note_num],2] = 1

    for (position, note_type, note_num, velocity) in notes:
        if position == normalized_num_steps:
            print('Warning: truncating from position {} to {}'.format(position, normalized_num_steps - 1))
            position = normalized_num_steps - 1
            # continue

        if position > normalized_num_steps:
            # print 'Warning: skipping note at position {} (greater than {})'.format(position, normalized_num_steps)
            continue
        if note_type == "note_on" and velocity > 0:
            open_msgs[note_num].append((position, note_type, note_num, velocity))
            # Note is hit
            midi_array[position, PITCHES_MAP[note_num],0] = 1
            midi_array[position, PITCHES_MAP[note_num],1] = 0
            midi_array[position, PITCHES_MAP[note_num],2] = 0
            velocity_array[position, PITCHES_MAP[note_num]] = velocity

    assert len(midi_array) == len(velocity_array)
    return midi_array, velocity_array

In [ ]:
def transform_2D_numpy_to3D_numpy(array):
  array_3D = np.zeros((len(array), len(array[0]), 3))
  for i in range(len(array)):
    for j in range(len(array[0])):
      
      is_hit = array[i][j][0] == 1
      is_played =  array[i][j][1] == 1

      if is_hit:
        array_3D[i][j][0] = 1
      elif is_played:
        array_3D[i][j][1] = 1
      else:
        array_3D[i][j][2] = 1
  
  return array_3D

In [ ]:
!unzip classical.zip

Archive:  jazz.zip
   creating: jazz/
  inflating: jazz/4thAvenueTheme.mid  
  inflating: jazz/A Sleepin' Bee.mid  
  inflating: jazz/accustomed.mid     
  inflating: jazz/afine-1.mid        
  inflating: jazz/afine-2.mid        
  inflating: jazz/Aghostofachance.mid  
  inflating: jazz/AHouseis.mid       
  inflating: jazz/Alabama.mid        
  inflating: jazz/alfiepno.mid       
  inflating: jazz/Aliceinw.mid       
  inflating: jazz/All The Things You Are.mid  
  inflating: jazz/AllOfMe2.mid       
  inflating: jazz/alltheth.mid       
  inflating: jazz/AllTheThings Reharmonized.mid  
  inflating: jazz/AllTheThings V2.mid  
  inflating: jazz/alone.mid          
  inflating: jazz/along.mid          
  inflating: jazz/angela.mid         
  inflating: jazz/Anthropology.mid   
  inflating: jazz/ARemarkYouMade.mid  
  inflating: jazz/Ask Me Now 2.mid   
  inflating: jazz/atIknew.mid        
  inflating: jazz/atrain2.mid        
  inflating: jazz/autleave.mid       
  inflating: jazz/Autu

In [ ]:
import os

In [ ]:
counter = 0
fail_count = 0
for filename in os.listdir("/content/classical"):
  try:
    mid = MidiFile( "/content/classical/" + filename)
    array = midi_to_array_one_hot(mid, 4)[0]
    array3D = transform_2D_numpy_to3D_numpy(array)
    save_path = "/content/classical3D_nice/classical_content_" + str(counter)
    np.save(save_path, array3D)
    counter +=1
  except:
    fail_count +=1

Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
1536
3072
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
2048
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
1024
8192
6144
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
6144
Bad MIDI, Note has no end time.
1024
8192
1536
8192
4096
Bad MIDI. Cannot have note off event before note on event
12288
8192
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
1536
4096
4096
3072
2048
Bad MIDI, Note has no end time.
1024
1024
1024
Bad MIDI. Cannot have note off event before note on event
256
Bad MIDI, Note has no end time.
6144
Bad MIDI. Cannot have note off event before note on event
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
768
Bad MIDI, Note has no end time.
Bad MIDI, Note has no end time.
384
1024
Bad MIDI, Note has no end time.
Bad MIDI, Note has no

In [1]:
!zip -r /content/classical3D.zip /content/classical3D